In [8]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "final"

filter_metric = "Objective/eval_all"
filter_operation = "min"

metrics = ["Objective/eval_forward", "Objective/eval_left", "Objective/eval_right", "eval_all_unified_depth/aux_depth_loss",
           "Objective/train"]

configs = ["config.VO.MODEL.visual_backbone", "config.VO.MODEL.pretrain_backbone",
           "config.VO.MODEL.cls_action", "config.VO.TRAIN.depth_aux_loss"]

step_threshold = 170

In [14]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api(timeout=45)
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, filter_metric, *metrics])

for run in tqdm.tqdm(runs):
    
    if run._state != "finished":
        continue
    
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    select = run_history["_step"]<step_threshold
    
    if filter_operation == "min":
        filter_metric_idx = run_history[filter_metric][select].argmin()
    elif filter_operation == "max":
        filter_metric_idx = run_history[filter_metric][select].argmax()
    else:
        break
    run_dict["best_run_path"] = run.config["config"]["CHECKPOINT_FOLDER"] + f"/ckpt_epoch_{filter_metric_idx}.pth"
    run_dict["best_run_idx"] = filter_metric_idx
        
    run_dict[filter_metric] = run_history[filter_metric][filter_metric_idx]
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        # catch models that only evaluate some actions
        if metric in run_history.keys():
            run_dict[metric] = run_history[metric][filter_metric_idx]
        else:
            run_dict[metric] = -1
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:10<00:00,  4.19it/s]


,name,id,state,config.VO.MODEL.visual_backbone,config.VO.MODEL.pretrain_backbone,config.VO.MODEL.cls_action,config.VO.TRAIN.depth_aux_loss,Objective/eval_all,Objective/eval_forward,Objective/eval_left,Objective/eval_right,eval_all_unified_depth/aux_depth_loss,Objective/train,best_run_idx,best_run_path
0,vo_vit_s_in21k_act_d,31ih4suq,finished,small,in21k,True,0,0.000878,0.000305,0.000486,0.000488,0,0.000048,132.0,train_log/final/vit/unique/vo_vit_s_in21k_act_...
1,vo_vit_s_dino_act_rgb_freeze,1gx1l0lj,finished,small,dino,True,0,0.015961,0.015409,0.004994,0.005074,0,0.015992,133.0,train_log/final/vit/unique/vo_vit_s_dino_act_r...
2,vo_vit_s_dino_act_rgb,2qnjbdgh,finished,small,dino,True,0,0.005505,0.005932,0.001414,0.001449,0,0.004173,38.0,train_log/final/vit/unique/vo_vit_s_dino_act_r...
3,vo_vit_s_dino_act_rgbd,3coc1q8f,finished,small,dino,True,0,0.000697,0.000289,0.000364,0.000365,0,0.000080,133.0,train_log/final/vit/unique/vo_vit_s_dino_act_r...
4,vo_vit_b_mmae_rgbd,1wfrlh4y,finished,base,mmae,False,0,0.000456,0.000168,0.000202,0.000242,0,0.000144,138.0,train_log/final/vit/unique/vo_vit_b_mmae_rgbd/...
5,vo_vit_s_dino_act_rgbd_freeze,3dnrym2s,finished,small,dino,True,0,0.015767,0.015031,0.005007,0.005103,0,0.015101,146.0,train_log/final/vit/unique/vo_vit_s_dino_act_r...
6,vo_vit_b_mmae_act_rgbd_freeze,uqk97nua,finished,base,mmae,True,0,0.014249,0.012987,0.004475,0.004544,0,0.011994,116.0,train_log/final/vit/unique/vo_vit_b_mmae_act_r...
7,vo_vit_s_dino_act_d,3duvi7ws,finished,small,dino,True,0,0.000834,0.000371,0.000442,0.000416,0,0.000163,145.0,train_log/final/vit/unique/vo_vit_s_dino_act_d...
8,vo_vit_s_dino_act_d_freeze,19i30twh,finished,small,dino,True,0,0.015337,0.014461,0.004969,0.004979,0,0.015061,148.0,train_log/final/vit/unique/vo_vit_s_dino_act_d...
9,vo_vit_b_mmae_act_rgbd,39bf0esq,finished,base,mmae,True,0,0.000513,0.000094,0.000330,0.000302,0,0.000047,70.0,train_log/final/vit/unique/vo_vit_b_mmae_act_r...


In [15]:
dt = datetime.datetime.now().strftime("%Y%m%d")
filename = f"experiments_2020_{dt}"

In [16]:
# # save df as csv file
# extension = ".csv"
# runs_df.to_csv(filename+extension)

In [17]:
!apt-get update -y
!apt-get install -y python-xlsxwriter
!pip install openpyxl

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,575 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]               
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-secur

     |████████████████████████████████| 242 kB 3.9 MB/s            


In [18]:
# save df as excel file
extension = ".xlsx"
with pd.ExcelWriter(filename+extension) as excel_writer:
    runs_df.to_excel(excel_writer, sheet_name='Sheet1')